In [3]:
import tensorflow as tf
import datetime
from tensorflow.keras import layers
from tensorflow.keras.layers import Input,Conv2D,MaxPooling2D,Flatten,Dense,Dropout
from tensorflow.keras import models
from tensorflow.keras.models import Model 
from tensorflow.keras import optimizers
from tensorflow.keras import callbacks
from tensorflow.keras import backend as K
from kerastuner.tuners import Hyperband
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical,plot_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.losses import MeanSquaredError
import os, shutil
#from PIL import Image, ImageDraw, ImageFont, ImageColor
import random 
import pandas as pd
import pydot
from tensorflow.keras.utils import Sequence
#from datagenerator import DataGenerator

In [4]:
train_dataset_dir ='\\Users\\danie\\Penn_State_REU_Jupyter\\Hexagon ML Project GitHub\\Training_set_6_25' 

df=pd.read_csv(os.path.join(train_dataset_dir,"training_dataframe.csv"))
#df = df.sample(frac=0.001,random_state=5).reset_index(drop=True)

df.shape

(300000, 18)

In [13]:
class DataGenerator(Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels,directory=None, batch_size=32, dim=(32,32), shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.directory= directory
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        yield X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim))
        y = np.empty((self.batch_size,8))

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            #x=np.load(os.path.join(self.directory,ID))
            X[i,] = np.reshape(np.load(os.path.join(self.directory,ID)),(self.dim[0],self.dim[1],1))

            # Store class
            y[i] = self.labels[ID]

        return X, y

In [14]:
validation_split_frac=0.2
batch_size=256
array_dim=(32,32,1)

train_partition=df['filename'][:int(df.shape[0]*validation_split_frac)]
val_partition=df['filename'][int(df.shape[0]*validation_split_frac):]
partition={"train":train_partition,'validation':val_partition}

#Creates a dictionary with keys as the data filenames and entries as the labels
label_names=['x_coord','y_coord','neighbor1x','neighbor1y','neighbor2x','neighbor2y','neighbor3x','neighbor3y']
ID_list=df['filename'].values
label_list=df[label_names].values
labels={}
i=0
for ID in ID_list:
    labels[ID]=label_list[i]
    i+=1

training_generator=DataGenerator(partition['train'],labels,shuffle=True,directory=train_dataset_dir,batch_size=batch_size,dim=array_dim)
validation_generator=DataGenerator(partition['validation'],labels,shuffle=True,directory=train_dataset_dir,batch_size=batch_size,dim=array_dim)

In [15]:
im=training_generator.__getitem__(1)[0][1]
print(im)

TypeError: 'generator' object is not subscriptable